In [10]:
from rocketcea.cea_obj_w_units import CEA_Obj
from rocketcea.cea_obj import add_new_fuel
import numpy as np
from scipy.optimize import minimize, shgo

In [2]:
allvectors = []               # array for all design vecs, global variable
allobjfun = []                # array for tracking objective function evaluations

In [35]:
def objective(var, cons):
    return (var/cons)**2 / 2
def exterior(var, cons, good_if_less_than=False):
    if good_if_less_than:
        return np.max([0, var/cons - 1])**2 / 2
    else:
        return np.max([0, -(var/cons - 1)])**2 / 2
def exact(var, cons):
    return (var/cons - 1)**2 / 2
def proportion(amount, ratio):
    top = amount * ratio/(1 + ratio)
    bottom = amount * 1/(1 + ratio)
    return top, bottom

In [84]:
def get_propellant_properties(alc_wt, of_ratio, p_ch, exp_ratio):
    ipa_wt = min(alc_wt, 99)
    eth_ratio = 21.11
    alc_wt = min(alc_wt, 99.5)
    eth_wt1, eth_wt2 = proportion(alc_wt, eth_ratio)
    ipa_str = '''
    fuel C3H8O-2propanol C 3 H 8 O 1    wt%=''' + str(ipa_wt) + '''
    h,cal=-65133.     t(k)=298.15   rho,g/cc=0.786
    fuel water H 2 O 1  wt%=''' + str(100 - ipa_wt) + '''
    h,cal=-68308.  t(k)=298.15 rho,g/cc=0.9998
    '''
    eth_str = '''
    fuel C2H5OH(L)   C 2 H 6 O 1       wt%=''' + str(eth_wt1) + '''
    h,cal=-66370.0      t(k)=298.15       rho,g/cc=0.789
    fuel C3H8O-2propanol C 3 H 8 O 1    wt%=''' + str(eth_wt2) + '''
    h,cal=-65133.     t(k)=298.15   rho,g/cc=0.786
    fuel water H 2 O 1  wt%=''' + str(100 - alc_wt) + '''
    h,cal=-68308.  t(k)=298.15 rho,g/cc=0.9998
    '''
    add_new_fuel('LV4_Fuel', ipa_str)
    #add_new_fuel('LV4_Fuel', eth_str)
    PROPELLANT = CEA_Obj(oxName='LOX', fuelName='LV4_Fuel',
                        pressure_units='Pa', temperature_units='K', cstar_units='m/s',
                         density_units='kg/m^3', isp_units='sec', specific_heat_units='J/kg-K')
    isp = PROPELLANT.get_Isp(Pc=p_ch, MR=of_ratio, eps=exp_ratio)
    IspVac, Cstar, Tc, MW, gamma = PROPELLANT.get_IvacCstrTc_ChmMwGam(Pc=p_ch, MR=of_ratio, eps=exp_ratio)
    return [isp, IspVac, Cstar, Tc, MW, gamma]

def prop_cost(x):
    ipa_wt, of_ratio, p_ch, exp_ratio = x
    #ipa_wt = 64.8
    isp, IspVac, Cstar, Tc, MW, gamma = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    merit = -objective(IspVac, 250) + 50* exterior(p_ch, 2413166, True) + 50 * exact(exp_ratio, 5)
    return merit

def prop_opt(x0):
    #res = minimize(prop_cost, x0, method='nelder-mead', options={'disp': True, 'adaptive':True})
    res = shgo(prop_cost,# options={'disp':True},
               n=50, iters=1, sampling_method='sobol',
               bounds=[*zip([0.1,0.1,0.1, 3],
                                [100,10,2413166*2,7])],
              minimizer_kwargs={'method':'Nelder-Mead', 'options':{'adaptive':True}})
    x = res.x
    return x

def prop_run():
    x = prop_opt([64.8, 1.3, 2413166, 5.988])
    ipa_wt, of_ratio, p_ch, exp_ratio = x
    ipa_wt = min(ipa_wt, 99)
    #ipa_wt = 64.8
    isp, IspVac, Cstar, Tc, MW, gamma = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    if __name__ == '__main__' and not '__file__' in globals():
        print('Alcohol Wt %: ', ipa_wt)
        print('OF ratio: ', of_ratio)
        print('P_ch (Pa): ', p_ch)
        print('Expansion ratio: ', exp_ratio)
        print()
        print('Vacuum ISP (s): ', IspVac)
        print('Chamber Temp (K): ', Tc)
        print('Molar Wt (1/n): ', MW)
        print('Spec Heat: ', gamma)
    global PROPELLANT_SET
    PROPELLANT_SET = True
    return ipa_wt, of_ratio, p_ch, Tc, MW, gamma
    

In [85]:
if __name__ == '__main__' and not '__file__' in globals(): prop_run()

reading cea isp data files for LOX / LV4_Fuel 67500 times
reading cea isp data files for LOX / LV4_Fuel 67600 times
reading cea isp data files for LOX / LV4_Fuel 67700 times
reading cea isp data files for LOX / LV4_Fuel 67800 times
reading cea isp data files for LOX / LV4_Fuel 67900 times
reading cea isp data files for LOX / LV4_Fuel 68000 times
reading cea isp data files for LOX / LV4_Fuel 68100 times
reading cea isp data files for LOX / LV4_Fuel 68200 times
reading cea isp data files for LOX / LV4_Fuel 68300 times
reading cea isp data files for LOX / LV4_Fuel 68400 times
reading cea isp data files for LOX / LV4_Fuel 68500 times
reading cea isp data files for LOX / LV4_Fuel 68600 times
reading cea isp data files for LOX / LV4_Fuel 68700 times
reading cea isp data files for LOX / LV4_Fuel 68800 times
reading cea isp data files for LOX / LV4_Fuel 68900 times
reading cea isp data files for LOX / LV4_Fuel 69000 times
reading cea isp data files for LOX / LV4_Fuel 69100 times
reading cea is

In [81]:
if __name__ == '__main__' and not '__file__' in globals(): prop_run()

reading cea isp data files for LOX / LV4_Fuel 63700 times
reading cea isp data files for LOX / LV4_Fuel 63800 times
reading cea isp data files for LOX / LV4_Fuel 63900 times
reading cea isp data files for LOX / LV4_Fuel 64000 times
reading cea isp data files for LOX / LV4_Fuel 64100 times
reading cea isp data files for LOX / LV4_Fuel 64200 times
reading cea isp data files for LOX / LV4_Fuel 64300 times
reading cea isp data files for LOX / LV4_Fuel 64400 times
reading cea isp data files for LOX / LV4_Fuel 64500 times
reading cea isp data files for LOX / LV4_Fuel 64600 times
reading cea isp data files for LOX / LV4_Fuel 64700 times
reading cea isp data files for LOX / LV4_Fuel 64800 times
reading cea isp data files for LOX / LV4_Fuel 64900 times
reading cea isp data files for LOX / LV4_Fuel 65000 times
reading cea isp data files for LOX / LV4_Fuel 65100 times
reading cea isp data files for LOX / LV4_Fuel 65200 times
reading cea isp data files for LOX / LV4_Fuel 65300 times
reading cea is